In [5]:
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [3]:
data_dir = Path.home() / 'Desktop/kaggle/data/quora'

train_all = pd.read_csv(data_dir / 'train.csv.zip')
test_all = pd.read_csv(data_dir / 'test.csv.zip')

print("Train shape : ", train_all.shape)
print("Test shape : ", test_all.shape)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


In [6]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = Counter()
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            vocab[word] += 1
    return vocab

In [7]:
sentences = train_all["question_text"].progress_apply(lambda x: x.split()).values
vocab = build_vocab(sentences)

100%|██████████| 1306122/1306122 [00:05<00:00, 236151.41it/s]


In [10]:
vocab.most_common(5)

[('the', 653948),
 ('What', 417802),
 ('to', 403183),
 ('a', 402682),
 ('in', 363131)]

In [11]:
import gensim

ImportError: No module named 'gensim'